In [1]:
import openai
from qdrant_client import QdrantClient

### Embedding function

In [2]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )
    return response.data[0].embedding

### Retrieval function

In [3]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [4]:
def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }

In [5]:
retrieved_context = retrieve_data("What kind of earphones can I get?", qdrant_client, k=10)

In [6]:
retrieved_context

{'retrieved_context_ids': ['B0BRV29L6C',
  'B09Y3PTLK3',
  'B09ZQ4FG91',
  'B09SKNHGQ5',
  'B09V7G7YY4',
  'B09T5BB1X3',
  'B0BCNYBQ38',
  'B09PR5GG4C',
  'B0B9S436PJ',
  'B0B9GM9MF9'],
 'retrieved_context': ['MORMOQUE T2 Wireless Gaming Earbuds, 45ms Low Latency Combat Mode, Wireless Earbuds with Microphone, IPX5 Waterproof，with 30H Playtime Gaming Mode-Gaming Wireless Headphones with Ultra-Low Latency; Hi-Fidelity Stereo Sound quality with Punchy Bass; Massive 30Hrs Total Playtime with the Charging Case; Touch Control Ergonomic Earbuds Battery-The massive battery power of 30 hrs playtime lets you explore all your entertainment without worrying about putting them on charge. Fast charging-Type-C super-fast charging gives you 100 mins of playtime in just 10 mins of charging. You don’t have to impatiently wait for long anymore. ENC Mic-In-line built microphone with ENC function which will reduce back ground noise and transfer clear voice during talking Breath LED light-Breath LED lights 

### Format retrieved context function

In [7]:
def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context

In [8]:
preprocessed_context = process_context(retrieved_context)

In [9]:
print(preprocessed_context)

- ID: B0BRV29L6C, rating: 4.0, description: MORMOQUE T2 Wireless Gaming Earbuds, 45ms Low Latency Combat Mode, Wireless Earbuds with Microphone, IPX5 Waterproof，with 30H Playtime Gaming Mode-Gaming Wireless Headphones with Ultra-Low Latency; Hi-Fidelity Stereo Sound quality with Punchy Bass; Massive 30Hrs Total Playtime with the Charging Case; Touch Control Ergonomic Earbuds Battery-The massive battery power of 30 hrs playtime lets you explore all your entertainment without worrying about putting them on charge. Fast charging-Type-C super-fast charging gives you 100 mins of playtime in just 10 mins of charging. You don’t have to impatiently wait for long anymore. ENC Mic-In-line built microphone with ENC function which will reduce back ground noise and transfer clear voice during talking Breath LED light-Breath LED lights decrates the headphones amazing and high-end
- ID: B09Y3PTLK3, rating: 2.9, description: YLYXCCH Audio Adapter for Wireless Bluetooth Xbox One Stereo Headset Audio Ad

### Create Prompt function

In [10]:
def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt

In [11]:
prompt = build_prompt(preprocessed_context, "What kind of earphones can I get?")

In [12]:
print(prompt)


You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
- ID: B0BRV29L6C, rating: 4.0, description: MORMOQUE T2 Wireless Gaming Earbuds, 45ms Low Latency Combat Mode, Wireless Earbuds with Microphone, IPX5 Waterproof，with 30H Playtime Gaming Mode-Gaming Wireless Headphones with Ultra-Low Latency; Hi-Fidelity Stereo Sound quality with Punchy Bass; Massive 30Hrs Total Playtime with the Charging Case; Touch Control Ergonomic Earbuds Battery-The massive battery power of 30 hrs playtime lets you explore all your entertainment without worrying about putting them on charge. Fast charging-Type-C super-fast charging gives you 100 mins of playtime in just 10 mins of charging. You don’t have to impatiently wait for long anymore. ENC Mic-In-line built micr

### Generate answer function

In [13]:
def generate_answer(prompt):

    response = openai.chat.completions.create(
        model="gpt-5-nano",
        messages=[{"role": "system", "content": prompt}],
        reasoning_effort="minimal"
    )

    return response.choices[0].message.content

In [14]:
print(generate_answer(prompt))

From the available products, you can get the MORMOQUE T2 Wireless Gaming Earbuds. They are wireless earbuds with a microphone, IPX5 waterproof rating, and features like ultra-low latency (45ms), up to 30 hours of total playtime with the charging case, Type-C fast charging, ENC noise reduction, and touch controls.


### Combined RAG Pipeline

In [15]:
def rag_pipeline(question, top_k=5):

    qdrant_client = QdrantClient(url="http://localhost:6333")

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    return answer

In [17]:
print(rag_pipeline("What kind of earphones can I get with ratings above 3.5?"))

Here are earphones in the available products with ratings above 3.5:

- MORMOQUE T2 Wireless Gaming Earbuds (ID B0BRV29L6C) – rating 4.0
  - Wireless earbuds with microphone, IPX5, 30H total playtime, Type-C fast charging, ENC mic.

- Turtle Beach Stealth 600 Gen 2 MAX Wireless Multiplatform Headset (ID B09ZQ4FG91) – rating 4.2
  - Wireless gaming headset with 48+ hour battery, 2.4GHz wireless, amplified audio, comfortable design.

If you’re specifically looking for earphones (as opposed to a headset), the MORMOQUE T2 earbuds fit your criterion with a rating above 3.5.
